In [1]:
import re
import unicodedata
import nltk
import string
import gensim
from nltk.corpus import stopwords
from collections import Counter
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical

nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize

import unicodedata
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
import pandas as pd
import numpy as np
import string

from time import time
from gensim.models import Word2Vec

from transformers import pipeline

ModuleNotFoundError: No module named 'nltk'

Section B: Text Preprocessing and Word Embedding (40 Marks)

Question 2: Text Preprocessing (20 Marks)

Given the dataset reviews.csv, perform the following preprocessing steps:

(a) Load the dataset and display the first 5 rows. (5 Marks)

(b) Remove any duplicate reviews. (5 Marks)

(c) Clean the text by removing punctuation, converting to lowercase, and removing stopwords. (10 Marks)

### 2.

#### (a) Load the dataset and display the first 5 rows. 

In [17]:
import pandas as pd

In [19]:
# Load the dataset
df = pd.read_csv('reviews.csv')
df.head()

,Review,Sentiment
0,This product exceeded my expectations! It's hi...,Positive
1,"The product was decent. It worked fine, but it...",Neutral
2,I had a terrible experience with this company....,Negative
3,It's an okay product. Nothing to write home ab...,Neutral
4,Disappointed with the product. It didn't meet ...,Negative


#### (b) Remove any duplicate reviews. 

In [21]:
df.shape

(386, 2)

In [23]:
df = df.drop_duplicates(subset='Review')

In [25]:
df.shape

(131, 2)

#### (c) Clean the text by removing punctuation, converting to lowercase, and removing stopwords.

In [27]:
# converting to lowercase
df['Review'] = df['Review'].str.lower()

In [29]:
# removing punctuations
import string

df['Review'] = df['Review'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [39]:
# removing stopwords
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\PRIYA
[nltk_data]     DAS/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
df['Review'] = df['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

Question 3: Word Embedding (20 Marks)

(a) Convert the cleaned text into word embeddings using TF-IDF(10 Marks)

(b) Display the shape of the resulting TF-IDF matrix. (4 Marks)

(c) Get the vocabulary of TF-IDF  (6 marks)



Solution:

#### (a) Convert the cleaned text into word embeddings using TF-IDF

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), stop_words= 'english')

df_tfidf_vectors = tfidf_vectorizer.fit_transform(df['Review'])

#### (b) Display the shape of the resulting TF-IDF matrix

In [49]:
df_tfidf_vectors.shape

(131, 147)

#### (c) Get the vocabulary of TF-IDF 

In [51]:
vocab = tfidf_vectorizer.vocabulary_
print(list(vocab.keys()))  # this prints all words in vocab

['product', 'exceeded', 'expectations', 'highquality', 'performs', 'exceptionally', 'decent', 'worked', 'fine', 'wasnt', 'special', 'terrible', 'experience', 'company', 'customer', 'service', 'rude', 'unhelpful', 'okay', 'write', 'home', 'disappointed', 'didnt', 'meet', 'avoid', 'costs', 'products', 'subpar', 'outstanding', 'exactly', 'looking', 'works', 'perfectly', 'absolutely', 'horrendous', 'topquality', 'satisfied', 'purchase', 'im', 'extremely', 'dissatisfied', 'cheaply', 'doesnt', 'function', 'properly', 'satisfactory', 'recommend', 'overpriced', 'work', 'advertised', 'exceptional', 'representatives', 'friendly', 'helpful', 'great', 'quality', 'average', 'particularly', 'good', 'bad', 'poor', 'worth', 'price', 'highly', 'blow', 'away', 'recommended', 'fantastic', 'knowledgeable', 'excellent', 'mediocre', 'job', 'awful', 'expected', 'item', 'money', 'worst', 'ive', 'received', 'faulty', 'unresponsive', 'neutral', 'stand', 'better', 'impressive', 'horrible', 'atrocious', 'meh', 'r

Section B: Model Building for Sentiment Analysis (40 Marks)

Question 4: Naive Bayes Classifier (20 Marks)

(a) Split the dataset into training and testing sets. (5 Marks)

(b) Train a Naive Bayes classifier on the training set. (10 Marks)

(c) Evaluate the model on the testing set and display the accuracy. (5 Marks)

Solution

#### (a) Split the dataset into training and testing sets. (5 Marks)

In [53]:
y = df['Sentiment']

In [55]:
from sklearn.model_selection import train_test_split

In [1]:
X_train, X_test, y_train, y_test = train_test_split(df_tfidf_vectors, y, test_size= 0.2, random_state= 42, stratify= y)

NameError: name 'train_test_split' is not defined

#### (b) Train a Naive Bayes classifier on the training set. 

In [59]:
from sklearn.naive_bayes import MultinomialNB

In [61]:
nb_classifier = MultinomialNB()
nb_model = nb_classifier.fit(X_train, y_train)

#### (c) Evaluate the model on the testing set and display the accuracy.

In [63]:
nb_predict = nb_model.predict(X_test)

In [65]:
from sklearn.metrics import accuracy_score

In [67]:
nb_accuracy = accuracy_score(y_test, nb_predict)
nb_accuracy

0.9629629629629629

Question 5: LSTM Model (20 Marks)

(a) Convert the cleaned text into sequences using Tokenizer. (5 Marks)

(b) Build and compile an LSTM model for sentiment analysis. (10 Marks)

(c) Train the model and evaluate its performance on the testing set. (5 Marks)

Solution:

#### (a) Convert the cleaned text into sequences using Tokenizer.

In [74]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Encode Sentiment to 0 and 1
le = LabelEncoder()
df['Sentiment'] = le.fit_transform(df['Sentiment'])

# 2. Tokenize Review text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['Review'])

# 3. Convert to sequences
sequences = tokenizer.texts_to_sequences(df['Review'])

# 4. Pad the sequences
padded_sequences = pad_sequences(sequences, padding='post', maxlen=100)

#### (b) Build and compile an LSTM model for sentiment analysis. 

In [87]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=64, input_length=100))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 classes

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#### (c) Train the model and evaluate its performance on the testing set.

In [78]:
from sklearn.model_selection import train_test_split

# Split padded sequences and encoded labels
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, df['Sentiment'], test_size=0.2, stratify=df['Sentiment'], random_state=42
)

# Train the model
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("LSTM Test Accuracy:", accuracy)

Epoch 1/5
4/4 [==============================] - 7s 611ms/step - loss: 0.6625 - accuracy: 0.3173 - val_loss: 0.6020 - val_accuracy: 0.3333
Epoch 2/5
4/4 [==============================] - 0s 112ms/step - loss: 0.5692 - accuracy: 0.3173 - val_loss: 0.4804 - val_accuracy: 0.3333
Epoch 3/5
4/4 [==============================] - 0s 96ms/step - loss: 0.4257 - accuracy: 0.3173 - val_loss: 0.2107 - val_accuracy: 0.3333
Epoch 4/5
4/4 [==============================] - 0s 101ms/step - loss: 0.1155 - accuracy: 0.3173 - val_loss: 0.0323 - val_accuracy: 0.3333
Epoch 5/5
1/1 [==============================] - 0s 65ms/step - loss: 0.0118 - accuracy: 0.3333
LSTM Test Accuracy: 0.3333333432674408
